# Lab 4. Energy Agent Collaborator

## Introduction

In previous notebook we have shown you how to create multi-agent collaborator feature on Amazon Bedrock.

[Multi-agent Collaboration](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) is a Amazon Bedrock Agents native capability that enables a hierarchical collaboration between agents. You can now enable agent collaboration and associate secondary agents to a supervisor one. These secondary agents can be any existing agent within the same account, including agents that have collaboration themselves. 

In this notebook we will interact with our supervisor agent created in the previous lab.

The following represents the complete architecture of multi-agent collaborator.

![Architecture](img/energy_manager_agent.png)

On this lab you will execute a set of tests, using detailed trace, to check the chain-of-thought reasoning that is happening on the agents.

**[Chain-of-thought reasoning](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-prompt-engineering.html)** is a prompt engineering technique that give step-by-step reasoning on how an answer is derived based on your prompt.


With Amazon Bedrock Agents you can use [traces](https://docs.aws.amazon.com/bedrock/latest/userguide/trace-events.html) to understand the step-by-step reasoning process of your agent. With the multi-agent collaboration feature, your supervisor trace also provides the reasoning details of its sub-agents

## Setup

Make sure that your boto3 version is the latest one.

If not, return no [notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) and run Setup block again.

In [ ]:
!pip freeze | grep boto3

## Invoking multi-agent Collaborator

Let's now invoke our supervisor agent with different queries to trigger the different sub-agents invocation. Before doing that, we're adding `bedrock_agent_helper.py` on Python path, so the file can be recognized and invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:

- `invoke`: invoking an agent with a query

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

curr_month = datetime.now()

In [ ]:
import sys
import uuid
import time

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

Before continuing, let's recover the information from our supervisor agent and it's sub-agents from our environment.

In [ ]:
%store -r
print("Supervisor agent name:", energy_agent_name, "id:", energy_agent_id)
print("Forecast agent name:", forecast_agent_name, "id:", forecast_agent_id, "alias_id:", forecast_agent_alias_id)
print("Solar Panel agent name:", solar_agent_name, "id:", solar_agent_id, "alias_id:", solar_agent_alias_id)
print("Peak Management agent name:", peak_agent_name, "id:", peak_agent_id, "alias_id:", peak_agent_alias_id)

We can now set a dictionary with agents names for better visualization of the traces

In [ ]:
multi_agent_names = {
    f"{forecast_agent_id}/{forecast_agent_alias_id}": forecast_agent_name,
    f"{solar_agent_id}/{solar_agent_alias_id}": solar_agent_name,
    f"{peak_agent_id}/{peak_agent_alias_id}": peak_agent_name,
    f"{energy_agent_id}/{energy_agent_alias_id}": energy_agent_name
}

On following section, you will invoke sub-agents but by multi-agent collaborator.

### Lab 1 - Forecasting Agent
First let's query our agent with prompts requiring the forecasting agent

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "제 예상 에너지 소비량을 확인하고, 현재 에너지 소비량과 비교해주세요. 제 고객 ID는 1입니다.", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "제 과거 에너지 소비량을 확인해주세요. 그리고 겨울철 평균 소비량을 알려주세요. 제 고객 ID는 1입니다.", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

future_2m = curr_month + relativedelta(months=2)
future_2m_formatted = future_2m.strftime("%Y/%m")

response = agents.invoke(
    f"{future_2m_formatted}에 대한 예측 소비량을 업데이트해주세요. 여행을 갈 예정이라 예측 소비량은 50입니다. 제 고객 ID는 1입니다.", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "제 월별 예상 에너지 소비량을 알려주세요. 제 고객 ID는 1입니다.", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

#### Lab 2 - Solar Panel Agent

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
response = agents.invoke(
    "제 Sunpower double-X 태양광 패널의 전기 이용방법이 규정을 준수하는지 어떻게 확인할 수 있나요?", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "제가 요청한 모든 티켓 정보를 알려주세요. 제 고객 ID는 1입니다.", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

#### Lab 3 - Peak Load Manager Agent

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "어떤 항목에서 많은 에너지 소비가 일어나고 있나요? 제 고객 ID는 2입니다.", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "제 소비량을 최적화할 수 있나요? 제 고객 ID는 1입니다.", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

### Next Steps

Congratulations! You've completed the workshop! 

By now you have created 3 sub-agents and a supervisor agent. You have invoked the supervisor agent using prompts requiring multiple sub-agents.

For other multi-agent collaboration examples check the [Amazon Bedrock Agent Samples](https://github.com/awslabs/amazon-bedrock-agent-samples) repository 

Next, let's clean up our resources to avoid unexpected costs